In [8]:
import numpy as np
import tensorflow as tf
import itertools as it
import scipy.io.wavfile as wavfile

tf.enable_eager_execution()

#testData = tf.data.TFRecordDataset("data/nsynth-test.tfrecord")
res = tf.python_io.tf_record_iterator("data/nsynth-test.tfrecord")

i = 0
for e in res:
    print("\nRECORD ", i, "\n")
    e = tf.train.Example.FromString(e)
    d = dict(e.features.feature)
    keys = d.keys()
    for key in keys:
        if key == 'audio':
            print(key)
            print('omitted\n')
            #print(d.get(key))
            #wavfile.read(d.get(key))
        else:
            print(key)
            print (d.get(key))
    
    i += 1
    if i > 0:
        break


RECORD  0 

qualities
int64_list {
  value: 0
  value: 0
  value: 0
  value: 0
  value: 0
  value: 0
  value: 0
  value: 0
  value: 0
  value: 0
}

instrument_source_str
bytes_list {
  value: "synthetic"
}

audio
omitted

instrument
int64_list {
  value: 417
}

qualities_str
bytes_list {
}

note
int64_list {
  value: 149013
}

instrument_str
bytes_list {
  value: "bass_synthetic_033"
}

instrument_family_str
bytes_list {
  value: "bass"
}

velocity
int64_list {
  value: 100
}

pitch
int64_list {
  value: 100
}

instrument_family
int64_list {
  value: 0
}

note_str
bytes_list {
  value: "bass_synthetic_033-100-100"
}

sample_rate
int64_list {
  value: 16000
}

instrument_source
int64_list {
  value: 2
}



In [10]:
import wave, sys, pyaudio

def play_wav(wf):
    paud = pyaudio.PyAudio()
    chunk = 1024
    stream = paud.open(format = paud.get_format_from_width(wf.getsampwidth()),
                    channels = wf.getnchannels(),
                    rate = wf.getframerate(),
                    output = True)
    data = wf.readframes(chunk)
    while data != '':
        stream.write(data)
        data = wf.readframes(chunk)

In [ ]:
#get number of records in test dataset
num_test_records = 0
ds_to_count = tf.data.TFRecordDataset('data/nsynth-test.tfrecord')
for r in ds_to_count:
    num_test_records += 1
    
print('num_test_records: ', num_test_records)

In [ ]:
#     features = {
#         "note_str": tf.FixedLenFeature([], dtype=tf.string),
#         "pitch": tf.FixedLenFeature([1], dtype=tf.int64),
#         "velocity": tf.FixedLenFeature([1], dtype=tf.int64),
#         "audio": tf.FixedLenFeature([64000], dtype=tf.float32),
#         "qualities": tf.FixedLenFeature([10], dtype=tf.int64),
#         "instrument_source": tf.FixedLenFeature([1], dtype=tf.int64),
#         "instrument_family": tf.FixedLenFeature([1], dtype=tf.int64),
#     }

In [1]:
from io import BytesIO, StringIO
from scipy.io import wavfile
import tensorflow as tf
from utils import audio_utils
import math,array
from magenta.models.nsynth import utils
import importlib

importlib.reload(audio_utils)

with tf.Graph().as_default():
    with tf.Session() as sess:
        # Read data
        dataset = tf.data.TFRecordDataset('data/nsynth-test.tfrecord')

        # make the tensor structure
        feats = {
            "note_str": tf.FixedLenFeature([], dtype=tf.string),
            "audio": tf.FixedLenFeature([64000], dtype=tf.float32)
        }

        parse_func = lambda example_proto: tf.parse_single_example(example_proto, feats)
        dataset = dataset.map(parse_func)

        num_samples = 50
        batch_size = 6
        dataset = dataset.shuffle(buffer_size=num_test_records).take(num_samples)
        dataset = dataset.batch(batch_size=batch_size)
        itr = dataset.make_one_shot_iterator()
        
        batch = itr.get_next()
        cnt = 0
        
        #loop through all batches
        for i in range(math.ceil(num_samples/batch_size)):
            ret = sess.run(batch)
            #print('iteration ', i)
            audio_utils.graph_wav(ret['audio'], ret['note_str'], cnt)
            #io = BytesIO(r)
            #print(io.read(10))
            #print(io)
            #rate, audio = wavfile.read(r)
            #play_wav(audio)
            #print(r)
            cnt += 1

/anaconda3/envs/magenta/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/envs/magenta/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/envs/magenta/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/envs/magenta/lib/python3.7/site-packages/tensorflow/python/framework

NameError: name 'num_test_records' is not defined